In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [7]:
import urllib.parse as urlparse
url='https://www.timeanddate.com/weather/india/chennai/historic?month=&year='

In [8]:
page=requests.get(url)
print(page)

<Response [200]>


In [9]:
month=range(1,13)
year=range(2009, 2024)
def insertchar(mystring,month,year):
    mystring=mystring[:65]+year+mystring[65:71]+month+mystring[71:]
    return mystring
result_url=[]
for i in year:
    for j in month:
        result_url.append(insertchar(url,str(i),str(j)))
        
result_url

['https://www.timeanddate.com/weather/india/chennai/historic?month=1&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=2&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=3&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=4&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=5&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=6&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=7&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=8&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=9&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=10&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=11&year=2009',
 'https://www.timeanddate.com/weather/india/chennai/historic?month=12&year=2009',
 'https://www.timeanddate

In [10]:
#to get  content of web for each date
for idx, url in enumerate(result_url):
    months_weather = requests.get(str(url))
    soup = BeautifulSoup(months_weather.content, 'html.parser')

    with open(r'C:\Users\KRITHIKA\Soupfile_{}.txt'.format(idx), 'w', encoding='utf-8') as f:
        f.write(str(soup))


In [11]:
for idx in range(0, 13):
    with open("C:/Users/KRITHIKA/Soupfile_{}.txt".format(idx), encoding='utf-8') as f:
        soup = f.read()
        bs = BeautifulSoup(soup, "html.parser")


In [12]:
'''from googletrans import Translator

translator = Translator()
translation = translator.translate('शनिवार 6 जुलाई 2024, 18.00 — 00.00', src='hi', dest='en')
print(translation.text)
'''

Saturday 6 July 2024, 18.00 - 00.00


In [20]:
import json
import pandas as pd
from bs4 import BeautifulSoup

dfs = pd.DataFrame()
for idx in range(0, 13):
    ddf_rows = []
    with open(r"C:/Users/KRITHIKA/Soupfile_{}.txt".format(idx), encoding='utf-8') as f:
        r = f.read()
    soup = BeautifulSoup(r, "html.parser")
    scripts = soup.find_all('script')
    jsonObj = None
    for script in scripts:
        if 'var data=' in script.text:
            jsonStr = script.text.strip()
            jsonStr = jsonStr.split('var data=')[1]
            jsonStr = jsonStr.split(';')[0]
            jsonObj = json.loads(jsonStr)
    if jsonObj:  # Ensure jsonObj is not None
        for item in jsonObj['detail']:
            date = item.get('ds')
            date_id = item.get('date')
            wind = item.get('wind')
            desc = item.get('desc')
            temp = item.get('temp')
            baro = item.get('baro')
            hum = item.get('hum')
            ddf_rows.append({'date': date, 'date_id': date_id, 'wind': wind, 'desc': desc, 'temp': temp, 'baro': baro, 'hum': hum})
    if ddf_rows:  # Only create DataFrame if there are rows to add
        ddf = pd.DataFrame(ddf_rows, columns=['date', 'date_id', 'wind', 'desc', 'temp', 'baro', 'hum'])
        dfs = pd.concat([dfs, ddf], axis=0).reset_index(drop=True)

# Print the first 5 rows
print(dfs.head(5))


                                  date       date_id  wind  \
0  रविवार 14 जुलाई 2024, 18.00 — 00.00  1.720980e+12     9   
1  सोमवार 15 जुलाई 2024, 00.00 — 06.00  1.721002e+12    14   
2  सोमवार 15 जुलाई 2024, 06.00 — 12.00  1.721023e+12    17   
3  सोमवार 15 जुलाई 2024, 12.00 — 18.00  1.721045e+12    13   
4  सोमवार 15 जुलाई 2024, 18.00 — 00.00  1.721066e+12    19   

                      desc  temp  baro  hum  
0            Drizzle. Fog.  30.0  1003   82  
1                     Fog.  29.0  1002   80  
2  Drizzle. Broken clouds.  29.0  1003   82  
3  Drizzle. Broken clouds.  29.0  1001   85  
4                     Fog.  29.0  1002   84  


In [25]:
hindi_to_english_months = {
    'जनवरी': 'January',
    'फरवरी': 'February',
    'मार्च': 'March',
    'अप्रैल': 'April',
    'मई': 'May',
    'जून': 'June',
    'जुलाई': 'July',
    'अगस्त': 'August',
    'सितंबर': 'September',
    'अक्टूबर': 'October',
    'नवंबर': 'November',
    'दिसंबर': 'December'
}

hindi_to_english_days = {
    'सोमवार': 'Monday',
    'मंगलवार': 'Tuesday',
    'बुधवार': 'Wednesday',
    'गुरुवार': 'Thursday',
    'शुक्रवार': 'Friday',
    'शनिवार': 'Saturday',
    'रविवार': 'Sunday'
}
def translate_date(text, translation_dict):
    for hindi, english in translation_dict.items():
        text = text.replace(hindi, english)
    return text

# Apply the translation function to the 'date' column
dfs['date'] = dfs['date'].apply(lambda x: translate_date(x, hindi_to_english_months))
dfs['date'] = dfs['date'].apply(lambda x: translate_date(x, hindi_to_english_days))

# Print the translated DataFrame
print(dfs)


                                         date       date_id  wind  \
0          Sunday 14 July 2024, 18.00 — 00.00  1.720980e+12     9   
1          Monday 15 July 2024, 00.00 — 06.00  1.721002e+12    14   
2          Monday 15 July 2024, 06.00 — 12.00  1.721023e+12    17   
3          Monday 15 July 2024, 12.00 — 18.00  1.721045e+12    13   
4          Monday 15 July 2024, 18.00 — 00.00  1.721066e+12    19   
...                                       ...           ...   ...   
1063  Saturday 30 January 2010, 18.00 — 00.00  1.264874e+12    10   
1064    Sunday 31 January 2010, 00.00 — 06.00  1.264896e+12     0   
1065    Sunday 31 January 2010, 06.00 — 12.00  1.264918e+12     7   
1066    Sunday 31 January 2010, 12.00 — 18.00  1.264939e+12    16   
1067    Sunday 31 January 2010, 18.00 — 00.00  1.264961e+12     7   

                         desc  temp  baro  hum  
0               Drizzle. Fog.  30.0  1003   82  
1                        Fog.  29.0  1002   80  
2     Drizzle. Broken cl

In [30]:
hindi_to_english_months = {
    'जनवरी': 'January',
    'फरवरी': 'February',
    'मार्च': 'March',
    'अप्रैल': 'April',
    'मई': 'May',
    'जून': 'June',
    'जुलाई': 'July',
    'अगस्त': 'August',
    'सितंबर': 'September',
    'अक्टूबर': 'October',
    'नवंबर': 'November',
    'दिसंबर': 'December'
}

hindi_to_english_days = {
    'सोमवार': 'Monday',
    'मंगलवार': 'Tuesday',
    'बुधवार': 'Wednesday',
    'गुरुवार': 'Thursday',
    'शुक्रवार': 'Friday',
    'शनिवार': 'Saturday',
    'रविवार': 'Sunday'
}

def translate_date(text):
    for hindi, english in hindi_to_english_months.items():
        text = text.replace(hindi, english)
    for hindi, english in hindi_to_english_days.items():
        text = text.replace(hindi, english)
    return text

# Reading and processing files
dft = pd.DataFrame()
for idx in range(0, 13):
    with open(r"C:/Users/KRITHIKA/Soupfile_{}.txt".format(idx), encoding='utf-8') as f:
        r = f.read()
    soup = BeautifulSoup(r, "html.parser")
    table = soup.find('table', id='wt-his')
    df = pd.DataFrame(columns=['time', 'weather', 'Humidity', 'wind', 'temp'])
    
    # Collecting Data
    for row in table.tbody.find_all('tr'):
        # Find all data for each column
        time = [sd.get_text() for sd in row.select("th")][0]
        weather = [t.get_text() for t in row.find_all('td', class_="small")][0]
        wind = [w.get_text() for w in row.find_all('td', class_="sep")][0]
        Humidity = [wh.get_text() for wh in row.find_all('td', class_="sep")][1]
        temp = [tp.get_text() for tp in row.find_all('td', class_="")][0]
        
        new_row = pd.DataFrame([{
            'time': time,
            'weather': weather,
            'Humidity': Humidity,
            'wind': wind,
            'temp': temp
        }])
        
        df = pd.concat([df, new_row], ignore_index=True)
    
    dft = pd.concat([dft, df], ignore_index=True)

dft['time'] = dft['time'].apply(translate_date)


dft.head(15)


,time,weather,Humidity,wind,temp
0,20.30Monday 29 July,Fog.,1002 mbar,9 km/h,31 °C
1,20.00,Fog.,1002 mbar,7 km/h,31 °C
2,19.30,Fog.,1001 mbar,19 km/h,31 °C
3,19.00,Broken clouds.,1001 mbar,20 km/h,31 °C
4,18.30,Broken clouds.,1000 mbar,19 km/h,32 °C
5,18.00,Broken clouds.,1000 mbar,19 km/h,32 °C
6,17.30,Broken clouds.,999 mbar,19 km/h,33 °C
7,16.30,Broken clouds.,999 mbar,20 km/h,35 °C
8,16.00,Broken clouds.,999 mbar,19 km/h,36 °C
9,15.30,Broken clouds.,999 mbar,19 km/h,36 °C


In [39]:
import pandas as pd
from bs4 import BeautifulSoup

# Sample translation dictionaries
hindi_to_english_months = {
    'जनवरी': 'January',
    'फरवरी': 'February',
    'मार्च': 'March',
    'अप्रैल': 'April',
    'मई': 'May',
    'जून': 'June',
    'जुलाई': 'July',
    'अगस्त': 'August',
    'सितंबर': 'September',
    'अक्टूबर': 'October',
    'नवंबर': 'November',
    'दिसंबर': 'December'
}

hindi_to_english_days = {
    'सोमवार': 'Monday',
    'मंगलवार': 'Tuesday',
    'बुधवार': 'Wednesday',
    'गुरुवार': 'Thursday',
    'शुक्रवार': 'Friday',
    'शनिवार': 'Saturday',
    'रविवार': 'Sunday'
}

def translate_date(text):
    if not isinstance(text, str):
        return text
    for hindi, english in hindi_to_english_months.items():
        text = text.replace(hindi, english)
    for hindi, english in hindi_to_english_days.items():
        text = text.replace(hindi, english)
    return text

# Reading and processing files
dfss = pd.DataFrame()
for idx in range(0, 13):
    with open(r"C:/Users/KRITHIKA/Soupfile_{}.txt".format(idx), encoding='utf-8') as f:
        r = f.read()
    soup = BeautifulSoup(r, "html.parser")
    table = soup.find('table', id='wt-his')
    dff = pd.read_html(str(table))[0]
    
    # Dropping a level down
    dff.columns = dff.columns.droplevel(0)
    dff = dff.drop(['Unnamed: 1_level_1', 'Unnamed: 5_level_1', 'Visibility'], axis=1)
    dfss = pd.concat([dfss, dff], axis=0).reset_index(drop=True)


# Translate Hindi words to English in all relevant columns
for column in dfss.columns:
    if dfss[column].dtype == 'object':
        dfss[column] = dfss[column].apply(translate_date)

# Print the first 5 rows after translation
dfss.head(15)


,Time,Temp,Weather,Wind,Humidity,Barometer
0,20.30 Monday 29 July,31 °C,Fog.,9 km/h,79%,1002 mbar
1,20.00,31 °C,Fog.,7 km/h,79%,1002 mbar
2,19.30,31 °C,Fog.,19 km/h,75%,1001 mbar
3,19.00,31 °C,Broken clouds.,20 km/h,75%,1001 mbar
4,18.30,32 °C,Broken clouds.,19 km/h,71%,1000 mbar
5,18.00,32 °C,Broken clouds.,19 km/h,71%,1000 mbar
6,17.30,33 °C,Broken clouds.,19 km/h,67%,999 mbar
7,16.30,35 °C,Broken clouds.,20 km/h,60%,999 mbar
8,16.00,36 °C,Broken clouds.,19 km/h,56%,999 mbar
9,15.30,36 °C,Broken clouds.,19 km/h,53%,999 mbar


In [41]:
dfss.to_csv(r'C:/Users/KRITHIKA/Weather_chennai_2.csv', sep=',')
dfs.to_csv(r'C:/Users/KRITHIKA/Weather_chennai_1.csv', sep=',')
dft.to_csv(r'C:/Users/KRITHIKA/Weather_chennai_3.csv', sep=',')